In [1]:
#Fundamentos em Big Data
#Tarefa 05
#Grupo 03
#05/02/2021

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder \
                    .appName("Tarefa_05") \
                    .config("spark.some.config.option", "some-value") \
                    .getOrCreate()

In [4]:
#Criando dataframes a partir das tabelas CSVs
df15 = spark.read.csv('DNBA2015.csv', header=True, encoding='ISO-8859-1')
df16 = spark.read.csv('DNBA2016.csv', header=True, encoding='ISO-8859-1')
df17 = spark.read.csv('DNBA2017.csv', header=True, encoding='ISO-8859-1')
df18 = spark.read.csv('DNBA2018.csv', header=True, encoding='ISO-8859-1')
df19 = spark.read.csv('DNBA2019.csv', header=True, encoding='ISO-8859-1')

In [5]:
#Criando dataframe único
df = df15.unionByName(df16)
df = df.unionByName(df17)
df = df.unionByName(df18)
df = df.unionByName(df19)

In [6]:
#Selecionando apenas as colunas de interese
df = df.select('CODMUNNASC', 'IDADEMAE', 'SEXO', 'DTNASC').orderBy('DTNASC')
#Criando coluna com o ano de nascimento
df = df.withColumn('ANO', F.substring(F.col('DTNASC'), 5,4))
#Código de Salvador=2905701
df=df.filter(F.col('CODMUNNASC') == '290570')

In [7]:
#Base para Cliente: nascidos vivos por ano
df_final = df.groupBy('ANO').count().orderBy('ANO')
df_final = df_final.withColumnRenamed('count', 'NASCIDOS')

In [8]:
#Base para Cliente: nascidos vivos do sexo masculino por ano
df2=df.where(F.col('SEXO')==1).groupBy('ANO').count().orderBy('ANO')
df2 = df2.withColumnRenamed('count', 'NASC_MASC')
df2 = df2.withColumnRenamed('ANO','ANO1')
df_final = df_final.join(df2,df_final.ANO==df2.ANO1)
df_final=df_final.drop('ANO1')

In [9]:
#Base para Cliente: nascidos vivos do sexo feminino por ano
df2=df.where(F.col('SEXO')==2).groupBy('ANO').count().orderBy('ANO')
df2 = df2.withColumnRenamed('count', 'NASC_FEM')
df2 = df2.withColumnRenamed('ANO','ANO1')
df_final = df_final.join(df2,df_final.ANO==df2.ANO1)
df_final=df_final.drop('ANO1')

In [10]:
#Base para Cliente: idade média da mãe por ano
#Convertendo tipo dos dados
df = df.withColumn("IDADEMAETmp", df["IDADEMAE"].cast('long')).drop("IDADEMAE").withColumnRenamed("IDADEMAETmp", "IDADEMAE")
df2=df.groupBy('ANO').mean('IDADEMAE').orderBy('ANO')
df2 = df2.withColumnRenamed('avg(IDADEMAE)', 'IDADE_MEDIA_MAE')
df2 = df2.withColumnRenamed('ANO','ANO1')
df_final = df_final.join(df2,df_final.ANO==df2.ANO1)
df_final=df_final.drop('ANO1')

In [11]:
#Convertendo tipo dos dados da base final
df_final = df_final.withColumn("ANOTmp", df_final["ANO"].cast('integer')).drop("ANO").withColumnRenamed("ANOTmp", "ANO")
df_final = df_final.withColumn("NASCIDOSTmp", df_final["NASCIDOS"].cast('integer')).drop("NASCIDOS").withColumnRenamed("NASCIDOSTmp", "NASCIDOS")
df_final = df_final.withColumn("NASC_MASCTmp", df_final["NASC_MASC"].cast('integer')).drop("NASC_MASC").withColumnRenamed("NASC_MASCTmp", "NASC_MASC")
df_final = df_final.withColumn("NASC_FEMTmp", df_final["NASC_FEM"].cast('integer')).drop("NASC_FEM").withColumnRenamed("NASC_FEMTmp", "NASC_FEM")
df_final = df_final.withColumn("IDADE_MEDIA_MAETmp", df_final["IDADE_MEDIA_MAE"].cast('long')).drop("IDADE_MEDIA_MAE").withColumnRenamed("IDADE_MEDIA_MAETmp", "IDADE_MEDIA_MAE")

In [12]:
df_final.printSchema()

root
 |-- ANO: integer (nullable = true)
 |-- NASCIDOS: integer (nullable = false)
 |-- NASC_MASC: integer (nullable = false)
 |-- NASC_FEM: integer (nullable = false)
 |-- IDADE_MEDIA_MAE: long (nullable = true)



In [13]:
df_final.show()

+----+--------+---------+--------+---------------+
| ANO|NASCIDOS|NASC_MASC|NASC_FEM|IDADE_MEDIA_MAE|
+----+--------+---------+--------+---------------+
|2016|    3026|     1568|    1458|             25|
|2019|    2691|     1315|    1376|             26|
|2017|    3219|     1635|    1584|             25|
|2018|    3247|     1679|    1568|             26|
|2015|    3247|     1682|    1565|             25|
+----+--------+---------+--------+---------------+



In [15]:
#TABELA FINAL EXPORTADA PARA CSV
df_final.repartition(1).write.csv('Tarefa05_Grupo03.csv',header='True')